## OpenPredict examples

Example call to the OpenPredict Smart API accessible at https://openpredict.137.120.31.102.nip.io

In [1]:
import json 
import requests

predict_drugs = requests.get('https://openpredict.137.120.31.102.nip.io/v1/predict',
                     params={'entity':"DRUGBANK:DB00394"})
print(json.dumps( predict_drugs.json(), indent = 2))

  },
    {
      "disease": "OMIM:131200",
      "drug": "DRUGBANK:DB00394",
      "score": 0.4044759949
    },
    {
      "disease": "OMIM:115645",
      "drug": "DRUGBANK:DB00394",
      "score": 0.0249104276
    },
    {
      "disease": "OMIM:200170",
      "drug": "DRUGBANK:DB00394",
      "score": 0.0610378424
    },
    {
      "disease": "OMIM:188050",
      "drug": "DRUGBANK:DB00394",
      "score": 0.1063279251
    },
    {
      "disease": "OMIM:218200",
      "drug": "DRUGBANK:DB00394",
      "score": 0.050881868
    },
    {
      "disease": "OMIM:188030",
      "drug": "DRUGBANK:DB00394",
      "score": 0.8938239118
    },
    {
      "disease": "OMIM:205700",
      "drug": "DRUGBANK:DB00394",
      "score": 0.7746811498
    },
    {
      "disease": "OMIM:146850",
      "drug": "DRUGBANK:DB00394",
      "score": 0.8674331159
    },
    {
      "disease": "OMIM:150699",
      "drug": "DRUGBANK:DB00394",
      "score": 0.4152291749
    },
    {
      "disease": "OMIM:6016

In [2]:
# For a disease
predict_diseases = requests.get('https://openpredict.137.120.31.102.nip.io/v1/predict',
                     params={'entity':"OMIM:268220", 
                             'classifier':'OpenPredict OMIM-DrugBank'})
print(json.dumps( predict_diseases.json(), indent = 2))

866
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB01400",
      "score": 0.0908150919
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB01003",
      "score": 0.1813192203
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB00367",
      "score": 0.1443335862
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB00573",
      "score": 0.1782956563
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB00476",
      "score": 0.0793179372
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB00470",
      "score": 0.0585841458
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB00550",
      "score": 0.0670889082
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB00804",
      "score": 0.0613545833
    },
    {
      "disease": "OMIM:268220",
      "drug": "DRUGBANK:DB01148",
      "score": 0.0579968549
    },
    {
      "disease": "OM

## Translator identifiers resolution

Translator ID preferences:
* [For Disease](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L2853)
* [Chemical substance](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L3097) 
* [Drug](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L3119)

Resolve CURIEs using Translator API services: 

In [3]:
resolve_curies = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     params={'curie':["OMIM:268220", "DRUGBANK:DB00394", "HP:0007354", "HGNC:613"]})
print( json.dumps( resolve_curies.json(), indent = 2))

{
  "OMIM:604308": {
    "id": {
      "identifier": "MONDO:0011431",
      "label": "MASS syndrome"
    },
    "equivalent_identifiers": [
      {
        "identifier": "MONDO:0011431",
        "label": "MASS syndrome"
      },
      {
        "identifier": "OMIM:604308"
      },
      {
        "identifier": "ORPHANET:99715"
      },
      {
        "identifier": "UMLS:C1858556"
      },
      {
        "identifier": "MESH:C536030"
      }
    ],
    "type": [
      "disease",
      "named_thing",
      "biological_entity",
      "disease_or_phenotypic_feature"
    ]
  },
  "DRUGBANK:DB09145": {
    "id": {
      "identifier": "CHEBI:15377",
      "label": "water"
    },
    "equivalent_identifiers": [
      {
        "identifier": "CHEBI:15377",
        "label": "water"
      },
      {
        "identifier": "CHEMBL.COMPOUND:CHEMBL1098659",
        "label": "WATER"
      },
      {
        "identifier": "DRUGBANK:DB09145"
      },
      {
        "identifier": "PUBCHEM.COMPOUND:962"